### Install repo requirements & import packages

In [ ]:
from __future__ import annotations

import logging

import numpy as np
from numpy.random._generator import Generator
from scipy import special
import matplotlib.pyplot as plt
import pandas as pd
import time as time

from elfpy import DEFAULT_LOG_MAXBYTES, WEI
from elfpy.types import MarketActionType, MarketAction, Config
from elfpy.simulators import Simulator
from elfpy.agent import Agent
from elfpy.markets import Market
from elfpy.utils import sim_utils
import elfpy.utils.outputs as output_utils
import elfpy.utils.post_processing as post_processing
from elfpy.types import MarketState, StretchedTime, Config
from elfpy.utils.outputs import get_gridspec_subplots

### Setup experiment parameters

In [ ]:
def DSR_historical(num_dates=90):
    dsr = pd.read_csv('https://s3-sim-repo-0.s3.us-east-2.amazonaws.com/Data/HIST_DSR_D.csv', index_col=0, infer_datetime_format=True)
    dsr.index = pd.to_datetime(dsr.index)
    dsr = dsr.resample('D').mean()
    min_date = dsr.index.min()
    max_date = dsr.index.max()
    date_range = max_date - min_date
    new_date_range = min_date + date_range * np.linspace(0, 1, num_dates)
    dsr_new = dsr.reindex(new_date_range, method='ffill')
    dsr_new = dsr_new.reset_index(drop=True)
    return dsr_new["DAI_SAV_RATE"].to_list()

config = Config()
config.random_seed=123
config.base_asset_price=1

config.log_filename = "./hyperdrive.log" # Output filename for logging

config.log_level = "WARNING" # Logging level, should be in ["DEBUG", "INFO", "WARNING"]. ERROR to suppress all logging.
config.pricing_model_name = "Hyperdrive" # can be yieldspace or hyperdrive

config.num_trading_days = 90 # Number of simulated trading days, default is 180
config.num_position_days = config.num_trading_days # term length
config.num_blocks_per_day = 10 #7200 # Blocks in a given day (7200 means ~12 sec per block)
config.trade_fee_percent = 0.10 # fee percent collected on trades
config.redemption_fee_percent = 0.005 # 5 bps

num_agents = 100 # int specifying how many agents you want to simulate
trade_chance = 2/(config.num_trading_days*config.num_blocks_per_day) # on a given block, an agent will trade with probability `trade_chance`

config.target_pool_apr = 0.01 # target pool APR of the initial market after the LP
config.target_liquidity = 5_000_000 # target total liquidity of the initial market, before any trades

vault_apr_init = 0.0 # Initial vault APR
vault_apr_jump_size = 0.001 # Scale of the vault APR change (vault_apr (+/-)= jump_size)
vault_jumps_per_year = 0#4 # The average number of jumps per year
vault_apr_jump_direction = "random_weighted" # The direction of a rate change. Can be 'up', 'down', or 'random'.
vault_apr_lower_bound = 0.01 # minimum allowable vault apr
vault_apr_upper_bound = 0.01 # maximum allowable vault apr

config.vault_apr = DSR_historical(num_dates=config.num_trading_days)

Create experiment configuration

### Setup agents

In [ ]:
class RegularGuy(Agent):
    """
    Agent that randomly opens or closes longs or shorts
    """

    def __init__(self, rng: Generator, trade_chance: float, wallet_address: int, budget: int = 10_000) -> None:
        """Add custom stuff then call basic policy init"""
        self.trade_long = True  # default to allow easy overriding
        self.trade_short = True  # default to allow easy overriding
        self.trade_chance = trade_chance
        self.rng = rng
        self.last_think_time = None
        self.threshold = self.rng.normal(loc=0.04, scale=0.02)
        super().__init__(wallet_address, budget)

    def action(self, market: Market) -> list[MarketAction]:
        """Implement a random user strategy

        The agent performs one of four possible trades:
            [OPEN_LONG, OPEN_SHORT, CLOSE_LONG, CLOSE_SHORT]
            with the condition that close actions can only be performed after open actions

        The amount opened and closed is random, within constraints given by agent budget & market reserve levels

        Parameters
        ----------
        market : Market
            the trading market

        Returns
        -------
        action_list : list[MarketAction]
        """
        gonna_trade = self.rng.choice([True, False], p=[self.trade_chance, 1-self.trade_chance])
        gonna_trade = True # deterministic af
        if not gonna_trade:
            return []
        # User can always open a trade, and can close a trade if one is open
        available_actions = []
        short_list = [short.balance for short in self.wallet.shorts.values()]
        long_list = [long.balance for long in self.wallet.longs.values()]
        has_opened_short = bool(any((short_balance > 0 for short_balance in short_list)))
        has_opened_long = bool(any((long_balance > 0 for long_balance in long_list)))
        if market.rate > market.market_state.vault_apr + self.threshold:
            # we want to make rate to go UP, so BUY PTs
            if has_opened_short is True:
                available_actions = [MarketActionType.CLOSE_SHORT] # buy to close
            elif self.trade_long is True:
                available_actions+=[MarketActionType.OPEN_LONG] # buy to open
        else:
            # we want to make rate go DOWN, so SELL PTs
            if has_opened_long is True:
                available_actions = [MarketActionType.CLOSE_LONG]
            elif self.trade_short is True:
                available_actions+=[MarketActionType.OPEN_SHORT] # sell to open
        action_type = self.rng.choice(available_actions, size=1) # choose one random trade type
        PT_needed = abs(market.pricing_model.calc_bond_reserves(
            target_apr=market.market_state.vault_apr,
            time_remaining=position_duration,
            market_state=market.market_state,
        )-market.market_state.bond_reserves)/2
        amount_to_trade_base = min(100_000,PT_needed*market.spot_price) if PT_needed > 0 else 0
        amount_to_trade_pt = amount_to_trade_base/market.spot_price
        if action_type == MarketActionType.OPEN_SHORT:
            max_short = self.get_max_short(market)
            if max_short > WEI: # if max_short is greater than the minimum eth amount
                trade_amount = np.maximum(WEI, np.minimum(max_short, amount_to_trade_pt)) # WEI <= trade_amount <= max_short
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                ]
            else: # no short is possible
                action_list = []
        elif action_type == MarketActionType.OPEN_LONG:
            max_long = self.get_max_long(market)
            if max_long > WEI: # if max_long is greater than the minimum eth amount
                trade_amount = np.maximum(WEI, np.minimum(max_long, amount_to_trade_base))
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                ]
            else:
                action_list = []
        elif action_type == MarketActionType.CLOSE_SHORT:
            # short_time = self.rng.choice(list(self.wallet.shorts)) # pick a random short
            biggest_short = max(short_list)
            for key, value in self.wallet.shorts.items():
                if (value.balance >= biggest_short) or (value.balance >= amount_to_trade_pt):
                    short_time = key
                    break
            trade_amount = np.maximum(WEI, np.minimum(amount_to_trade_pt, self.wallet.shorts[short_time].balance))
            open_share_price = self.wallet.shorts[short_time].open_share_price
            action_list = [
                self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=short_time, open_share_price=open_share_price),
            ]
        elif action_type == MarketActionType.CLOSE_LONG:
            # long_time = self.rng.choice(list(self.wallet.longs)) # pick a random long
            biggest_long = max(long_list)
            for key, value in self.wallet.longs.items():
                if (value.balance >= biggest_long) or (value.balance >= amount_to_trade_pt):
                    long_time = key
                    break
            trade_amount = np.maximum(WEI, np.minimum(amount_to_trade_pt, self.wallet.longs[long_time].balance))
            action_list = [
                self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=long_time),
            ]
        else:
            action_list = []
        if action_list:
            print(
                f"t={market.time*365:.0f}: F:{market.rate:.3%}V:{market.market_state.vault_apr:.3%}"
                +f"agent #{self.wallet.address:03.0f} is going to {action_type} of size {trade_amount}",
                end=""
                )
            if self.last_think_time is not None:
                print(f"in {time.time() - self.last_think_time:.3f} seconds")
        self.last_think_time = time.time()
        return action_list


class LPAgent(Agent):
    """
    Adds a large LP
    """
    def action(self, market: Market):
        """
        implement user strategy
        LP if you can, but only do it once
        short if you can, but only do it once
        """
        if self.wallet.lp_tokens > 0: # has already opened the lp
            action_list = []
        else:
            action_list = [
                self.create_agent_action(
                    action_type=MarketActionType.ADD_LIQUIDITY, trade_amount=self.budget
                ),
            ]
        return action_list


def get_example_agents(rng: Generator, budget: float, new_agents: int, existing_agents: int = 0, direction: str = None) -> list[Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = RegularGuy(
            rng=rng,
            trade_chance=trade_chance,
            wallet_address=address,
            budget=budget,
        )
        if direction is not None:
            if direction == "short":
                agent.trade_long = False
            if direction == "long":
                agent.trade_short = False
        agent.log_status_report()
        agents += [agent]
    return agents

### Setup simulation objects

In [ ]:
# define root logging parameters
output_utils.setup_logging(
    log_filename=config.log_filename,
    log_level=output_utils.text_to_log_level(config.log_level),
)

# instantiate the pricing model
pricing_model = sim_utils.get_pricing_model(config.pricing_model_name)

# instantiate the market
position_duration = StretchedTime(
    days=config.num_position_days * 365,
    time_stretch=pricing_model.calc_time_stretch(config.target_pool_apr),
    normalizing_constant=config.num_position_days * 365,
)

init_target_liquidity = 1 # tiny amount for setting up apr
share_reserves_direct, bond_reserves_direct = pricing_model.calc_liquidity(
    market_state=MarketState(
        share_price=config.init_share_price,
        init_share_price=config.init_share_price
    ),
    target_liquidity=init_target_liquidity,
    target_apr=config.target_pool_apr,
    position_duration=position_duration,
)
market = Market(
    pricing_model=pricing_model,
    market_state=MarketState(
        share_reserves=share_reserves_direct,
        bond_reserves=bond_reserves_direct,
        base_buffer=0,
        bond_buffer=0,
        lp_reserves=init_target_liquidity / config.init_share_price,
        init_share_price=config.init_share_price,  # u from YieldSpace w/ Yield Baring Vaults
        share_price=config.init_share_price,  # c from YieldSpace w/ Yield Baring Vaults
        vault_apr=config.vault_apr[0],  # yield bearing source apr
        trade_fee_percent=config.trade_fee_percent,  # g
        redemption_fee_percent=config.redemption_fee_percent,
    ),
    position_duration=position_duration
)

# Instantiate the initial LP agent.
current_market_liquidity = market.pricing_model.calc_total_liquidity_from_reserves_and_price(
    market_state=market.market_state, share_price=market.market_state.share_price
)
lp_amount = config.target_liquidity - current_market_liquidity
init_agents = [LPAgent(wallet_address=0, budget=lp_amount)]

# initialize the simulator using only the initial LP.
simulator = Simulator(
    config=config,
    market=market,
)
simulator.add_agents(init_agents)
simulator.collect_and_execute_trades()


### Run the simulation

In [ ]:
# add the random agents
# short_agents = get_example_agents(rng=simulator.rng, budget=agent_budget, new_agents=num_agents, existing_agents=1, direction="short")
# long_agents = get_example_agents(rng=simulator.rng, budget=agent_budget, new_agents=num_agents, existing_agents=1+num_agents, direction="long")
# simulator.add_agents(short_agents + long_agents)
regular_guy = get_example_agents(rng=simulator.rng, budget=1_000_000_000, new_agents=1, existing_agents=1)
simulator.add_agents(regular_guy)
print(f"Simulator has {len(simulator.agents)} agents")

# run the simulation
simulator.run_simulation()

In [ ]:
# convert simulation state to a pandas dataframe
trades = post_processing.compute_derived_variables(simulator)
for col in trades:
    if col.startswith("agent") and not col.endswith("lp_tokens"):
        divisor = 1e6 # 1 million divisor for everyone
        trades[col] = trades[col] / divisor
print(f"number of trades = {len(trades)}")
display(trades.head(5))
print(trades.columns)

In [ ]:
# aggregate data
keep_columns = [
    "day",
]
trades_agg = trades.groupby(keep_columns).agg(
    {
        'spot_price': ['mean'],
        'delta_base_abs': ['sum','count'],
        'share_reserves': ['mean'],
        'bond_reserves': ['mean'],
        'lp_reserves': ['mean'],
        'agent_0_pnl_no_mock': ['mean'],
    }
)
trades_agg.columns = ["_".join(col).strip() for col in trades_agg.columns.values]
trades_agg = trades_agg.reset_index()
display(trades_agg.head(5))

### Plot simulation results

This shows the evolution of interest rates over time. The "vault" APR represents a theoretical underlying variable rate. Here we've mocked it up to have the same pattern as the MakerDao DAI Saving Rate over its whole history, but condensed to a 90 day period for this simulation. The fixed rate is initialized at 1% and appears to remain unchanged.

In [ ]:
trades.agent_0_lp_tokens

In [ ]:
exclude_first_trade = True
exclude_last_trade = True
fig, ax = plt.subplots(4,1, sharex=True, gridspec_kw={'wspace': 0.3, 'hspace': 0.0}, figsize=(10,10))
start_idx = 1 if exclude_first_trade is True else 0
first_trade_that_is_on_last_day = min(trades.index[trades.day == max(trades.day)])
end_idx = first_trade_that_is_on_last_day - 1 if exclude_last_trade is True else len(trades)

# first subplot
trades.iloc[start_idx:end_idx].plot(x="index", y=['share_reserves','bond_reserves','lp_reserves','agent_0_lp_tokens'], ax=ax[0])
ax[0].set_ylabel("# of tokens")
ax[0].get_lines()[3].set_linestyle("--")

# second subplot
ax[1] = trades.iloc[start_idx:end_idx].plot(x="index", y=['pool_apr','vault_apr'], ax=ax[1])
ax[1].legend(loc='best')

# third subplot
trades.iloc[start_idx:end_idx].plot(x="index", y=['spot_price'], ax=ax[2])

# fourth subplot
l = trades.iloc[start_idx:end_idx].plot(x="index", y=['agent_0_pnl_no_mock'], ax=ax[3])
# ax[3].set_yticklabels([f"{(i):.0%}" for i in ax[1].get_yticks()])
ax[3].set_xlabel("Trade")
r = trades.iloc[start_idx:end_idx].plot(x="index", y=['agent_1_pnl_no_mock'], ax=ax[3], secondary_y=True)
# axis = trades.iloc[start_idx:end_idx].plot(x="index", y=['agent_0_pnl_no_mock'], ax=ax[3], linestyle='dotted', color='k')
# axis = trades.iloc[start_idx:end_idx].plot(x="index", y=['agent_1_pnl_no_mock'], ax=ax[3], linestyle='dotted', color='k', secondary_y=True)
second_ax = ax[3].right_ax

plt.show()

These random agents are unable to pick smart entry points. Due to trading on coinflips only, they slowdly bleed fees out of their starting position, which in this case reduces from 1.0 million down to 0.999, a loss of $1k.

In [ ]:
def get_pnl_excluding_agent_0_no_mock_with_day(trades_df: pd.DataFrame) -> pd.DataFrame:
    """Returns Profit and Loss Column for every agent except for agent 0 from post-processing"""
    cols_to_return = ['day']+[col for col in trades_df if col.startswith("agent") and col.endswith("pnl_no_mock")]
    cols_to_return.remove("agent_0_pnl_no_mock")
    return trades_df[cols_to_return]

def plot_pnl(pnl, ax, label, last_day):
    # ax.plot(pnl.iloc[1:,:], linestyle='-', linewidth=0.5, alpha=0.5)
    # separate first half of agents, which are set to trade short
    # from second half of agents, which are set to trade long
    columns = pnl.columns.to_list()
    if len(columns)==1:
        ax.plot(pnl.iloc[1:,:], c='black', label=f"{label}, final_value={pnl.iloc[-1,0]:.5f}", linewidth=2)
    else:
        n = int(len(columns)/2)
        short_pnl = pnl.loc[1:, columns[:n]].mean(axis=1)
        long_pnl = pnl.loc[1:, columns[n:]].mean(axis=1)
        ax.plot(short_pnl, c='red', label=f"Short {label}, final value={short_pnl.iloc[-1,0]:.5f}", linewidth=2)
        ax.plot(long_pnl, c='black', label=f"Long {label}, final_value={long_pnl.iloc[-1,0]:.5f}", linewidth=2)
    # grey area where day is last day
    ax.set_ylabel('PNL in millions')
    # ax.axvspan(last_day, len(short_pnl), color='grey', alpha=0.2, label="Last day")
    ax.legend()

fig, ax = plt.subplots(1, 1, figsize=(6, 5), sharex=True, gridspec_kw={'wspace': 0.0, 'hspace': 0.0})
first_trade_that_is_on_last_day = min(trades.index[trades.day == max(trades.day)])
# data_mock = post_processing.get_pnl_excluding_agent_0(trades)
data_no_mock = get_pnl_excluding_agent_0_no_mock_with_day(trades).groupby('day').mean()
# plot_pnl(pnl=data_mock,label='Mock',ax=ax[0],last_day=first_trade_that_is_on_last_day)
plot_pnl(pnl=data_no_mock,label="Realized Market Value",ax=ax,last_day=first_trade_that_is_on_last_day)

xtick_step = 10
ax.set_xticks([0]+[x for x in range(9, config.num_trading_days + 1, xtick_step)])
ax.set_xticklabels(['1']+[str(x+1) for x in range(9, config.num_trading_days + 1, xtick_step)])

plt.gca().set_xlabel("Day")
plt.gca().set_title('Trader PNL over time')
# display(data_no_mock)
plt.show()

This plot shows being a Liquidity Provider (LP) is a profitable position, in this scenario where agents are trading randomly.

In [ ]:
display(trades_agg.columns)
fig, ax = plt.subplots(3,1,figsize=(6, 15))
exclude_last_day = False
exclude_first_day = False
num_agents = 1
start_idx = 0 if exclude_first_day is False else 1
first_trade_that_is_on_last_day = min(trades_agg.index[trades_agg.day == max(trades_agg.day)])
end_idx = len(trades_agg) - 2 if exclude_last_day is True else len(trades_agg)-1
data = trades_agg.loc[start_idx:end_idx,"agent_0_pnl_no_mock_mean"]

# first subplot
ax[0].plot(trades_agg.loc[start_idx:end_idx,"day"], data, label=f"mean = {trades_agg.loc[end_idx,'agent_0_pnl_no_mock_mean']:.3f}")
ax[0].set_title("LP PNL Over Time")
ax[0].set_ylabel("PNL")
ax[0].set_xlabel("Day")
xtick_step = 10
ax[0].set_xticks([0]+[x for x in range(9, config.num_trading_days + 1, xtick_step)])
ax[0].set_xticklabels(['1']+[str(x+1) for x in range(9, config.num_trading_days + 1, xtick_step)])
ax[0].legend({f"final value = {data.values[len(data)-1]:,.3f}"})
ax[0].set_ylabel("PnL in millions")

# second subplot
ax[1].bar(trades_agg.loc[start_idx:end_idx,"day"],\
    trades_agg.loc[start_idx:end_idx,"delta_base_abs_sum"],\
    label=f"mean = {trades_agg.loc[start_idx:end_idx,'delta_base_abs_sum'].mean():,.0f}")
ax[1].legend(loc='best')
ax[1].set_title("Market Volume")
ax[1].set_ylabel("Base")
ax[1].set_xlabel("Day")
xtick_step = 10
ax[1].set_xticks([0]+[x for x in range(9, config.num_trading_days + 1, xtick_step)])
ax[1].set_xticklabels(['1']+[str(x+1) for x in range(9, config.num_trading_days + 1, xtick_step)])
ylim = ax[1].get_ylim()
ax[1].set_ylim(0, ylim[1])

# third subplot
ax[2].bar(trades_agg.loc[start_idx:end_idx,"day"],\
    trades_agg.loc[start_idx:end_idx,"delta_base_abs_count"],\
    label=f"mean = {trades_agg.loc[start_idx:end_idx,'delta_base_abs_count'].mean():,.1f}")
ax[2].legend(loc='best')
ax[2].set_title("# of trades")
ax[2].set_xlabel("Day")
xtick_step = 10
ax[2].set_xticks([0]+[x for x in range(9, config.num_trading_days + 1, xtick_step)])
ax[2].set_xticklabels(['1']+[str(x+1) for x in range(9, config.num_trading_days + 1, xtick_step)])
ylim = ax[2].get_ylim()
ax[2].set_ylim(0, ylim[1])

plt.show()

## We are constantly updating our research. Stay tuned for more!

TODO:
- parameter optimization
- smart agents
- multiple simulation trial runs to evaluate LP profitability
- simulate Aave, Compound, MakerDao, etc.